In [1]:
import datetime
import random
import math
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal

%load_ext line_profiler


In [2]:
def make_5min_bars(tick_data):
    #Resample to get 5min bars
    five_min_data = pd.DataFrame(
        tick_data['last'].resample('5Min', loffset=datetime.timedelta(minutes=5)).ohlc())
        
    #Create RTH Calendar
        
    #We hack the NYSE Calendar extending the close until 4:15
    class CMERTHCalendar(mcal.exchange_calendar_nyse.NYSEExchangeCalendar):
        @property
        def close_time(self):
            return datetime.time(16, 15)
        
    nyse = CMERTHCalendar()
    print(nyse.tz.zone)
    schedule = nyse.schedule(start_date=five_min_data.index.min(), 
                             end_date=five_min_data.index.max())
        
    #Filter out those bars that occur during RTH
    five_min_data['dates'] = pd.to_datetime(five_min_data.index.to_datetime().date)
    five_min_data['valid_date'] = five_min_data['dates'].isin(schedule.index)
    five_min_data['valid_time'] = False
    during_rth = five_min_data['valid_date'] & \
        (five_min_data.index > schedule.loc[five_min_data['dates'],'market_open']) & \
        (five_min_data.index <= schedule.loc[five_min_data['dates'],'market_close'])
    five_min_data.loc[during_rth, 'valid_time'] = True
    five_min_data = five_min_data[five_min_data['valid_time'] == True]
    five_min_data.drop(['dates','valid_date','valid_time'], axis=1, inplace=True)
        
    #Add ema
    five_min_data['ema'] = five_min_data['close'].ewm(span=20, min_periods=20).mean()

    #Reset index
    five_min_data.reset_index(inplace=True)
        
    #Add column for number of seconds elapsed in trading day
    five_min_data['sec'] = (five_min_data['date'].values 
                            - five_min_data['date'].values.astype('datetime64[D]')) / np.timedelta64(1,'s')

    #Calculate sin & cos time
    #24hr time is a cyclical continuous feature
    seconds_in_day = 24*60*60
    five_min_data['sin_time'] = np.sin(2*np.pi*five_min_data['sec']/seconds_in_day)
    five_min_data['cos_time'] = np.cos(2*np.pi*five_min_data['sec']/seconds_in_day)

    five_min_data.drop('sec', axis=1, inplace=True)
        
    return five_min_data

In [20]:
td = pd.read_feather('../data/processed/ES_tick.feather')
td = td[td['date'] > '2017-07-29']
#td = td[td['date'] > '2018-01-24']
#Create Index from date column
td.index = td['date']
td.drop(labels=['date'],axis=1,inplace=True)
#Remove duplicates -- timestamps are equal!!!
td = td[~td.index.duplicated()]
td.head()
#td = pd.read_hdf('../data/processed/store.h5', key='tick_data')
#td.columns = ['last','volume']
#td.index.rename('date', inplace=True)
#td.index = td.index.tz_localize('US/Eastern')
#td = td[~td.index.duplicated()]
#td.head()

,last,bid,ask,volume
date,,,,
2017-07-30 18:00:00.041000-04:00,2471.50,2471.5,2471.50,1
2017-07-30 18:00:00.056000-04:00,2471.75,2471.5,2471.75,1
2017-07-30 18:00:00.063000-04:00,2471.50,2471.5,2471.75,1
2017-07-30 18:00:00.068000-04:00,2471.50,2471.5,2471.75,5
2017-07-30 18:00:00.093000-04:00,2471.75,2471.5,2471.75,9


In [20]:
td.index = td.index.tz_localize('America/Chicago')
td.head()

,last,volume
date,,
2006-01-02 17:00:02-06:00,1255.25,2
2006-01-02 17:00:04-06:00,1255.50,3
2006-01-02 17:00:05-06:00,1255.25,1
2006-01-02 17:00:08-06:00,1255.50,5
2006-01-02 17:00:09-06:00,1255.50,5


In [22]:
td.index = td.index.tz_convert('America/New_York')
td.head()

,last,volume
date,,
2006-01-02 18:00:02-05:00,1255.25,2
2006-01-02 18:00:04-05:00,1255.50,3
2006-01-02 18:00:05-05:00,1255.25,1
2006-01-02 18:00:08-05:00,1255.50,5
2006-01-02 18:00:09-05:00,1255.50,5


In [23]:
store = pd.HDFStore('../data/processed/store.h5')
store.put('processed_data',td)
store.close()

In [3]:
td = pd.read_hdf('../data/processed/store.h5', key='processed_data')
td.head()

,last,volume
date,,
2006-01-02 18:00:02-05:00,1255.25,2
2006-01-02 18:00:04-05:00,1255.50,3
2006-01-02 18:00:05-05:00,1255.25,1
2006-01-02 18:00:08-05:00,1255.50,5
2006-01-02 18:00:09-05:00,1255.50,5


In [4]:
td.shape

(192917901, 2)

In [21]:
fd = make_5min_bars(td)

#Add columns tracking reward for buy/sell actions and number of bars until reward
fd.loc[:,'btc'] = 0
fd.loc[:,'stc'] = 0

America/New_York


In [22]:
fd['close'].isnull().values.any()

False

In [23]:
fd.tail(100)

,date,open,high,low,close,ema,sin_time,cos_time,btc,stc
9905,2018-01-24 14:45:00-05:00,2841.25,2842.50,2841.00,2842.25,2838.815737,-0.896873,0.442289,0,0
9906,2018-01-24 14:50:00-05:00,2842.50,2845.25,2842.50,2844.00,2839.309476,-0.887011,0.461749,0,0
9907,2018-01-24 14:55:00-05:00,2844.25,2844.50,2841.75,2842.25,2839.589526,-0.876727,0.480989,0,0
9908,2018-01-24 15:00:00-05:00,2842.25,2843.00,2841.75,2842.50,2839.866714,-0.866025,0.500000,0,0
9909,2018-01-24 15:05:00-05:00,2842.50,2842.75,2839.50,2839.75,2839.855598,-0.854912,0.518773,0,0
9910,2018-01-24 15:10:00-05:00,2839.75,2841.50,2839.75,2841.25,2839.988398,-0.843391,0.537300,0,0
9911,2018-01-24 15:15:00-05:00,2841.25,2842.00,2839.00,2840.00,2839.989503,-0.831470,0.555570,0,0
9912,2018-01-24 15:20:00-05:00,2839.75,2841.25,2838.75,2840.00,2839.990503,-0.819152,0.573576,0,0
9913,2018-01-24 15:25:00-05:00,2839.75,2841.00,2839.25,2840.75,2840.062836,-0.806445,0.591310,0,0
9914,2018-01-24 15:30:00-05:00,2840.75,2841.50,2839.25,2840.50,2840.104471,-0.793353,0.608761,0,0


In [24]:
def simulate(start_index, action, order_price):
    position = 'flat'
    working_order = None
    reward = 0
    #for i, r in fd[start_index+1:].iterrows():
    for i in fd[start_index+1:start_index+82].index:
         # Get tick data for the current bar
        #ts_end = r['date']
        ts_end = fd.at[i, 'date']
        ts_start = ts_end - pd.Timedelta(minutes=5)
        ticks = td.loc[ts_start:ts_end, :]
        
        if position == 'flat':
            if action == 'buy':
                position = 'long'
                target_price = order_price + 2
                stop_price = order_price - 2
            elif action == 'sell':
                position = 'short'
                target_price = order_price - 2
                stop_price = order_price + 2
                
        
        #for t_i, t_r in ticks.iterrows():
        for t_i in ticks.index:
            price = ticks.at[t_i, 'last']
            if position == 'long':
                if price > target_price:
                    position = 'flat'
                    reward = 96
                elif price <= stop_price:
                    position = 'flat'
                    reward = -104
            elif position == 'short':
                if price < target_price:
                    position = 'flat'
                    reward = 96
                elif price >= stop_price:
                    position = 'flat'
                    reward = -104
        
        if position == 'flat':
            break
            
    return reward
    

In [18]:
%lprun -f simulate simulate(0, 'buy', fd.loc[0, 'low'])

In [12]:
fd.shape

(253323, 12)

In [15]:
td.index.dtype

datetime64[ns, US/Eastern]

In [25]:
%%time
for index, row in fd[:-1].iterrows():
    fd.loc[index, 'btc'] = simulate(index, 'buy', fd.at[index,'close'])
    fd.loc[index, 'stc'] = simulate(index, 'sell', fd.at[index,'close'])

CPU times: user 13min 7s, sys: 60 ms, total: 13min 7s
Wall time: 13min 7s


In [9]:
%%time
for index in fd[:-1].index:
    fd.loc[index, 'btc'] = simulate(index, 'buy', fd.at[index,'close'])
    fd.loc[index, 'stc'] = simulate(index, 'sell', fd.at[index,'close'])

KeyboardInterrupt: 

In [26]:
fd.head(100)

,date,open,high,low,close,ema,sin_time,cos_time,btc,stc
0,2017-07-31 09:35:00-04:00,2474.75,2475.75,2474.00,2475.50,NaN,-0.402747,-0.915311,-104,96
1,2017-07-31 09:40:00-04:00,2475.25,2476.00,2473.75,2475.50,NaN,-0.422618,-0.906308,-104,96
2,2017-07-31 09:45:00-04:00,2475.75,2475.75,2474.50,2474.75,NaN,-0.442289,-0.896873,-104,96
3,2017-07-31 09:50:00-04:00,2474.50,2475.00,2473.50,2473.75,NaN,-0.461749,-0.887011,-104,96
4,2017-07-31 09:55:00-04:00,2474.00,2474.25,2472.75,2472.75,NaN,-0.480989,-0.876727,-104,96
5,2017-07-31 10:00:00-04:00,2472.75,2473.25,2471.00,2471.25,NaN,-0.500000,-0.866025,-104,96
6,2017-07-31 10:05:00-04:00,2471.50,2471.75,2468.25,2468.75,NaN,-0.518773,-0.854912,96,-104
7,2017-07-31 10:10:00-04:00,2468.50,2470.75,2468.50,2470.50,NaN,-0.537300,-0.843391,-104,96
8,2017-07-31 10:15:00-04:00,2470.50,2470.75,2469.25,2470.50,NaN,-0.555570,-0.831470,-104,96
9,2017-07-31 10:20:00-04:00,2470.25,2471.25,2470.25,2470.50,NaN,-0.573576,-0.819152,-104,96


In [27]:
fd = fd[(fd['btc']!=0) & (fd['stc']!=0)]
fd.tail()

,date,open,high,low,close,ema,sin_time,cos_time,btc,stc
9996,2018-01-25 15:35:00-05:00,2838.00,2839.25,2836.50,2838.00,2838.033805,-0.779884,0.625923,96,-104
9997,2018-01-25 15:40:00-05:00,2838.00,2838.25,2836.25,2837.00,2837.935347,-0.766044,0.642788,96,-104
9998,2018-01-25 15:45:00-05:00,2837.00,2839.00,2836.25,2838.25,2837.965314,-0.751840,0.659346,96,-104
9999,2018-01-25 15:50:00-05:00,2838.25,2840.25,2838.00,2838.50,2838.016236,-0.737277,0.675590,96,-104
10000,2018-01-25 15:55:00-05:00,2838.50,2840.50,2838.25,2838.50,2838.062309,-0.722364,0.691513,96,-104


In [19]:
store = pd.HDFStore('../data/processed/store.h5')
store.put('cnn_data',fd)
store.close()

In [28]:
fd.to_feather('../data/processed/ES_TFCnn.feather')

In [13]:
fd = pd.read_feather('../data/processed/ES_5mintrading.feather')
fd.tail()

,date,open,high,low,close,ema,sin_time,cos_time,sell_r,sell_b,buy_r,buy_b,cdl_body,cdl_ut,cdl_lt,cdl_rng
10000,2018-01-25 15:55:00-05:00,2838.50,2840.50,2838.25,2838.50,2838.062309,-0.722364,0.691513,-50,3,0,0,0.00,2.00,0.25,2.25
10001,2018-01-25 16:00:00-05:00,2838.75,2840.75,2838.25,2839.75,2838.223042,-0.707107,0.707107,-50,2,0,0,1.00,1.00,0.50,2.50
10002,2018-01-25 16:05:00-05:00,2840.00,2841.00,2839.50,2840.00,2838.392276,-0.691513,0.722364,0,0,0,0,0.00,1.00,0.50,1.50
10003,2018-01-25 16:10:00-05:00,2840.25,2840.75,2840.00,2840.75,2838.616821,-0.675590,0.737277,-50,0,0,0,0.50,0.00,0.25,0.75
10004,2018-01-25 16:15:00-05:00,2840.75,2841.75,2840.50,2841.00,2838.843790,-0.659346,0.751840,0,0,0,0,0.25,0.75,0.25,1.25


In [34]:
fd['cdl_sign'] = np.sign(fd['close'] - fd['open'])
fd['cdl_body'] = np.absolute(fd['close'] - fd['open'])
#fd['cdl_ut'] = fd['high'] - fd['close'] if fd['cdl_body'] > 0 else fd['high'] - fd['open']
fd['cdl_ut'] = np.where(fd['cdl_sign'] > 0, fd['high'] - fd['close'], fd['high'] - fd['open'])
fd['cdl_lt'] = np.where(fd['cdl_sign'] > 0, fd['open'] - fd['low'], fd['close'] - fd['low'])
fd['cdl_rng'] = fd['high'] - fd['low']
fd['cdl_hl'] = np.where(fd['low'] >= fd['low'].shift(), 1, 0) #higher low
fd['cdl_lh'] = np.where(fd['high'] <= fd['high'].shift(), 1, 0) #lower high
fd.tail(10)

,date,open,high,low,close,ema,sin_time,cos_time,sell_r,sell_b,buy_r,buy_b,cdl_sign,cdl_body,cdl_ut,cdl_lt,cdl_rng,cdl_hl,cdl_lh
253316,2018-06-22 15:30:00-04:00,2764.00,2765.00,2763.75,2764.00,2764.390244,-0.923880,0.382683,0,0,-50,0,0.0,0.00,1.00,0.25,1.25,1,0
253317,2018-06-22 15:35:00-04:00,2764.00,2764.00,2762.50,2763.75,2764.329269,-0.915311,0.402747,0,0,0,0,-1.0,0.25,0.00,1.25,1.50,0,1
253318,2018-06-22 15:40:00-04:00,2763.50,2763.75,2762.50,2762.50,2764.155053,-0.906308,0.422618,0,0,50,1,-1.0,1.00,0.25,0.00,1.25,1,1
253319,2018-06-22 15:45:00-04:00,2762.75,2763.50,2762.25,2762.25,2763.973619,-0.896873,0.442289,-50,0,0,0,-1.0,0.50,0.75,0.00,1.25,0,1
253320,2018-06-22 15:50:00-04:00,2762.50,2765.00,2762.25,2763.50,2763.928513,-0.887011,0.461749,0,0,-50,1,1.0,1.00,1.50,0.25,2.75,1,0
253321,2018-06-22 15:55:00-04:00,2763.25,2764.50,2762.00,2762.50,2763.792464,-0.876727,0.480989,0,0,-50,0,-1.0,0.75,1.25,0.50,2.50,0,1
253322,2018-06-22 16:00:00-04:00,2762.25,2762.75,2755.25,2757.50,2763.193181,-0.866025,0.500000,0,0,0,0,-1.0,4.75,0.50,2.25,7.50,0,1
253323,2018-06-22 16:05:00-04:00,2757.75,2761.75,2757.50,2761.00,2762.984307,-0.854912,0.518773,0,0,0,0,1.0,3.25,0.75,0.25,4.25,1,1
253324,2018-06-22 16:10:00-04:00,2761.25,2761.50,2759.25,2759.50,2762.652468,-0.843391,0.537300,0,0,0,0,-1.0,1.75,0.25,0.25,2.25,1,1
253325,2018-06-22 16:15:00-04:00,2759.50,2760.75,2758.75,2759.50,2762.352233,-0.831470,0.555570,0,0,0,0,0.0,0.00,1.25,0.75,2.00,0,1


In [35]:
fd.shape

(253326, 19)

In [36]:
store = pd.HDFStore('../data/processed/store.h5')
store.put('final_data',fd)
store.close()

In [25]:
fd.to_feather('../data/processed/ES_5mintrading.feather')

In [26]:
td = pd.read_feather('../data/processed/ES_5mintrading.feather')
td.tail()

,date,open,high,low,close,ema,sin_time,cos_time,sell_r,sell_b,buy_r,buy_b,cdl_body,cdl_ut,cdl_lt,cdl_rng,cdl_sign,cdl_hl,cdl_lh
10000,2018-01-25 15:55:00-05:00,2838.50,2840.50,2838.25,2838.50,2838.062309,-0.722364,0.691513,-50,3,0,0,0.00,2.00,0.25,2.25,0.0,1,0
10001,2018-01-25 16:00:00-05:00,2838.75,2840.75,2838.25,2839.75,2838.223042,-0.707107,0.707107,-50,2,0,0,1.00,1.00,0.50,2.50,1.0,1,0
10002,2018-01-25 16:05:00-05:00,2840.00,2841.00,2839.50,2840.00,2838.392276,-0.691513,0.722364,0,0,0,0,0.00,1.00,0.50,1.50,0.0,1,0
10003,2018-01-25 16:10:00-05:00,2840.25,2840.75,2840.00,2840.75,2838.616821,-0.675590,0.737277,-50,0,0,0,0.50,0.00,0.25,0.75,1.0,1,1
10004,2018-01-25 16:15:00-05:00,2840.75,2841.75,2840.50,2841.00,2838.843790,-0.659346,0.751840,0,0,0,0,0.25,0.75,0.25,1.25,1.0,1,0
